In [1]:
from ultralytics import YOLO
import cv2
# Load a model
model = YOLO('models/yolov8x-pose-p6.pt', verbose=False)  # load an official model
# model = YOLO('path/to/best.pt')  # load a custom model

image = cv2.imread(r"D:\GitHub\Hackathon-Pune-Metro\1_avi\frame_0.jpg")
results = model(image)  # predict on an image

# for r in results:
#     print(r.keypoints.xy)  # print string of prediction results


0: 768x1280 1 person, 2552.4ms
Speed: 10.2ms preprocess, 2552.4ms inference, 1270.6ms postprocess per image at shape (1, 3, 768, 1280)


In [ ]:
import math

def get_angle(point1, point2, point3):
    x1, y1 = point1
    x2, y2 = point2
    x3, y3 = point3
    
    angle1 = math.atan2(y2 - y1, x2 - x1)
    angle2 = math.atan2(y3 - y2, x3 - x2)
    
    angle = math.degrees(angle2 - angle1)
    # angle = abs(angle)
    return angle

# Example usage
point1 = (0, 0)
point2 = (3, 3)
point3 = (6, 0)
angle = get_angle(point1, point2, point3)
print(angle)

-90.0


In [ ]:
video_path = 'dataset/escalator-fall/1.avi'
model = YOLO('models/yolov8x-pose-p6.pt', verbose=False)  # load an official model
results = model(video_path, stream=True)  # predict on an image

In [ ]:
import cv2
from ultralytics import YOLO
import pandas as pd
import numpy as np

# Load the YOLOv8 model
# model = YOLO('yolov8n.pt')
# Open the video file
frame_count = 0
threshold_angle = 50
df = pd.DataFrame(columns=["person_number", "frame_number", "left_angle", "right_angle"])

# Loop through the video frames
for frame in results:
    if frame_count % 1 == 0:  # Skip 10 frames
        neccessary_points = [12, 14, 16, 11, 13, 15]
        results = model(frame, stream=False)  # predict on an image
        print(frame_count)

        # Loop through all keypoints
        r=results[0]
        x_y_points = []
        for person_number in range(len(r.keypoints[0].xy)):
            if len(r.keypoints[0].xy[person_number]) != 17:
                print('keypoint empty')
                continue
            # Get the x and y coordinates of the keypoint
            for point_number in neccessary_points:
                x = int(r.keypoints[0].xy[person_number][point_number][0])
                y = int(r.keypoints[0].xy[person_number][point_number][1])
                if x == 0:
                    print(f'{frame_count}, empty, x:{x}, y:{y}')
                    x_y_points = []
                    break
                x_y_points.append((x, y))
                # Draw a circle at the keypoint location with the random color
                # cv2.circle(image2, (int(x), int(y)), 5, color, -1)
            if len(x_y_points) == 0:
                continue
            # print(x_y_points)
            right_angle = get_angle(x_y_points[0], x_y_points[1], x_y_points[2])
            left_angle = get_angle(x_y_points[3], x_y_points[4], x_y_points[5])
            df = pd.concat([df, pd.DataFrame({'person_number': [person_number], 'frame_number': [frame_count], 'left_angle': [left_angle], 'right_angle': [right_angle]})], ignore_index=True)
            # Append side and angle to df dataframe

    # Read the next frame
    frame_count += 1

# Release the video capture object and close the display window
df.to_csv('1_avi.csv')


0: 768x1280 1 person, 2472.1ms
Speed: 7.1ms preprocess, 2472.1ms inference, 1.0ms postprocess per image at shape (1, 3, 768, 1280)
0



C:\Users\dusng\AppData\Local\Temp\ipykernel_27100\657819765.py:49: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame({'person_number': [person_number], 'frame_number': [frame_count], 'left_angle': [left_angle], 'right_angle': [right_angle]})], ignore_index=True)


0: 768x1280 1 person, 2383.8ms
Speed: 9.1ms preprocess, 2383.8ms inference, 2.0ms postprocess per image at shape (1, 3, 768, 1280)
1

0: 768x1280 1 person, 2562.8ms
Speed: 12.0ms preprocess, 2562.8ms inference, 1.0ms postprocess per image at shape (1, 3, 768, 1280)
2

0: 768x1280 1 person, 2586.1ms
Speed: 12.1ms preprocess, 2586.1ms inference, 2.0ms postprocess per image at shape (1, 3, 768, 1280)
3



KeyboardInterrupt: 

In [2]:
import pandas as pd

df = pd.read_csv('1_avi.csv', index_col=0)
df_filtered = df[(abs(df['left_angle']) > 60) & (abs(df['right_angle']) > 60)]

df_filtered

,person_number,frame_number,left_angle,right_angle
64,0,110,-65.480357,-76.087505
65,0,111,-71.565051,-93.945186
66,0,114,-72.814294,-64.440035
67,0,115,-96.340192,-75.564025
68,0,116,-100.619655,-90.842524
...,...,...,...,...
1088,0,1344,-127.203009,-101.364604
1089,0,1345,-118.383354,-96.051732
1090,0,1346,-102.858090,-87.735245
1091,0,1347,-105.851928,-70.377744


In [ ]:
df = pd.read_csv('1_false_avi.csv', index_col=0)
df_filtered = df[(abs(df['left_angle']) > 60) & (abs(df['right_angle']) > 60)]

df_filtered